# Lung

In [1]:
setwd('/gpfs/gibbs/pi/zhao/xs272/prep_pipeline/')
library(Matrix)
library(readr)
library(purrr)
source('/gpfs/gibbs/pi/zhao/xs272/data_preprocess/utils.r')
source('/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/scripts/utils.r')

Attaching SeuratObject

Attaching sp

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Loading required package: ensembldb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:Matrix’:

    

## 1. Unimodal RNA preprocessing
### FACS

In [58]:
smart.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/RNA/FACS'
smart.rna.mat <- read.csv(file.path(smart.path, 'Lung-counts.csv'))

smart.rna.mat.mat <- as(data.matrix(smart.rna.mat[, -1]), 'dgCMatrix')

rownames(smart.rna.mat.mat) <- format.genes(smart.rna.mat[, 1])
colnames(smart.rna.mat.mat) <- colnames(smart.rna.mat)[-1]

In [59]:
smart.rna.mat <- smart.rna.mat.mat

In [32]:
any(duplicated(colnames(smart.rna.mat), colnames(smart.rna.mat)))

[1] FALSE

In [33]:
print.count.dim(smart.rna.mat)

[1] "No NA."
[1] "No NA rows."
[1] "No NA cols."
[1] "Dimension of count:"
[1] 23433  1923


In [34]:
facs.anno.file <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/RNA/annotations_facs_lung.csv'
facs.anno.table <- read.csv(facs.anno.file)

In [35]:
facs.count <- smart.rna.mat
print.names.match(colnames(facs.count), facs.anno.table$cell)

[1] "Length1, Length2, Length intersection, identical:"
[1] "1923 1676 1676 FALSE"


In [36]:
facs.count <- facs.count[, facs.anno.table$cell]
facs.anno <- facs.anno.table$annotation

In [37]:
print.count.dim(facs.count)

[1] "No NA."
[1] "No NA rows."
[1] "No NA cols."
[1] "Dimension of count:"
[1] 23433  1676


### droplet (10x)

In [38]:
droplet.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/RNA/droplet/'
droplet.folders <- dir(droplet.path, full.names = FALSE, pattern = 'Lung')

In [39]:
droplet.list <- map(droplet.folders, 
                    ~ list(
                        counts = readMM(file.path(droplet.path, .x, 'matrix.mtx')), 
                        barcodes = format.barcode(readLines(file.path(droplet.path, .x, 'barcodes.tsv'))), 
                        genes = format.genes(read.table(file.path(droplet.path, .x, 'genes.tsv'))[,1])
                        )
                   ) %>% setNames(droplet.folders)

In [40]:
length(droplet.list)

[1] 4

In [41]:
for (i in 1:length(droplet.list)) {
    for (j in 1:i) {
        print.names.match(droplet.list[[i]]$genes, droplet.list[[j]]$genes)
    }
}

[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"
[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"
[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"
[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"
[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"
[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"
[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"
[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"
[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"
[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"


In [42]:
fix.droplet <- function(tuple, name) {
    add_name <- gsub('.*(?=10X)', '', name, perl=TRUE)
    barcodes <- tuple$barcodes
    barcodes <- paste(add_name, barcodes, sep='_')
    counts <- tuple$counts
    genes <- tuple$genes
    colnames(counts) <- barcodes
    rownames(counts) <- genes
    counts
}

In [43]:
droplet.list.proc <- map2(droplet.list, names(droplet.list), 
                    fix.droplet
                   )

In [44]:
for (i in 1:length(droplet.list)) {
    for (j in 1:i) {
        print(identical(droplet.list[[i]]$genes, droplet.list[[j]]$genes))
    }
}

[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE


In [45]:
droplet.counts <- do.call(cbind, droplet.list.proc)

In [46]:
print.count.dim(droplet.counts)

[1] "No NA."
[1] "No NA rows."
[1] "No NA cols."
[1] "Dimension of count:"
[1]   23433 1476710


In [47]:
droplet.anno.file <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/RNA/annotations_droplet_lung.csv'
droplet.anno <- read.csv(droplet.anno.file)

In [48]:
print.names.match(colnames(droplet.counts), droplet.anno$cell)

[1] "Length1, Length2, Length intersection, identical:"
[1] "1476710 5404 5404 FALSE"


In [49]:
droplet.counts <- droplet.counts[, droplet.anno$cell]
droplet.anno <- droplet.anno$annotation

In [50]:
print.count.dim(droplet.counts)

[1] "No NA."
[1] "No NA rows."
[1] "No NA cols."
[1] "Dimension of count:"
[1] 23433  5404


In [61]:
print.names.match(rownames(droplet.counts), rownames(facs.count))

[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 FALSE"


In [62]:
droplet.counts <- droplet.counts[rownames(facs.count), ]

In [63]:
print.names.match(rownames(droplet.counts), rownames(facs.count))

[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 23433 23433 TRUE"


## 2. Unimodal Gene Activity (sci-ATAC-seq)

In [64]:
sci.ga.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/ATAC'
sci.ga.mat <- readMM(file.path(sci.ga.path, 'activity_scores.quantitative.mtx.gz'))
sci.ga.genes <- readLines(file.path(sci.ga.path, 'activity_scores.quantitative.genes.txt'))
sci.ga.cells <- readLines(file.path(sci.ga.path, 'activity_scores.quantitative.cells.txt'))

In [65]:
any(duplicated(format.genes(sci.ga.genes)))

[1] FALSE

In [66]:
sci.ga.mat <- set.names(counts = sci.ga.mat, rowname = format.genes(sci.ga.genes), colname = sci.ga.cells)

In [67]:
sci.ga.annot <- read.csv(file.path(sci.ga.path, 'cell_metadata.tissue_freq_filtered_lung.txt'))

In [68]:
any(duplicated(sci.ga.annot$cell))

[1] FALSE

In [69]:
unique(sci.ga.annot$tissue)

[1] "Lung"

In [70]:
print.names.match(sci.ga.annot$cell, colnames(sci.ga.mat))

[1] "Length1, Length2, Length intersection, identical:"
[1] "9924 81173 9924 FALSE"


In [71]:
sci.ga.mat <- sci.ga.mat[, sci.ga.annot$cell]

## 3. Align unimodal RNA and ATAC gene activities
gene names  
cell types  

In [72]:
print.names.match(rownames(facs.count), rownames(sci.ga.mat))

[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 20783 17165 FALSE"


In [74]:
print.names.match(rownames(droplet.counts), rownames(sci.ga.mat))

[1] "Length1, Length2, Length intersection, identical:"
[1] "23433 20783 17165 FALSE"


In [75]:
sort(unique(facs.anno))

[1] "B cell"             "Endothelial cell"   "Leukocyte"         
 [4] "Monocyte"           "Multiciliated cell" "Myeloid cell"      
 [7] "NK cell"            "Pneumocyte"         "Stromal cell"      
[10] "T cell"

In [76]:
sort(unique(droplet.anno))

[1] "Alveolar macrophage" "B cell"              "Endothelial cell"   
 [4] "Leukocyte"           "Monocyte"            "Multiciliated cell" 
 [7] "Myeloid cell"        "NK cell"             "Pneumocyte"         
[10] "Stromal cell"        "T cell"

In [77]:
sort(unique(sci.ga.annot$annotation))

[1] "Alveolar macrophage"      "B cell"                  
 [3] "Collision"                "Dendritic cell"          
 [5] "Endothelial cell"         "Hematopoietic progenitor"
 [7] "Macrophage"               "Monocyte"                
 [9] "NK cell"                  "Pneumocyte"              
[11] "T cell"                   "Unknown"

In [78]:
print.names.match(unique(facs.anno), unique(sci.ga.annot$annotation))

[1] "Length1, Length2, Length intersection, identical:"
[1] "10 12 6 FALSE"


In [79]:
print.names.match(unique(droplet.anno), unique(sci.ga.annot$annotation))

[1] "Length1, Length2, Length intersection, identical:"
[1] "11 12 7 FALSE"


In [81]:
facs.path <- 'lung/FACS/'
write.counts.transpose(facs.count, facs.path)
write.annotation(facs.anno, facs.path)

Warning message in dir.create(folder, recursive = TRUE):
"'lung/FACS' already exists"


In [82]:
droplet.path <- 'lung/droplet/'
write.counts.transpose(droplet.counts, droplet.path)
write.annotation(droplet.anno, droplet.path)

Warning message in dir.create(folder, recursive = TRUE):
"'lung/droplet' already exists"


In [83]:
sci.ga.path <- 'lung/sci-atac_gene_activities/'
write.counts.transpose(sci.ga.mat, sci.ga.path)
write.annotation(sci.ga.annot$annotation, sci.ga.path)

Warning message in dir.create(folder, recursive = TRUE):
"'lung/sci-atac_gene_activities' already exists"


## 4. Unimodal ATAC (sci-ATAC-seq)

In [84]:
atac.peak.path = 'tmp/sci_atac'
system(paste0('mkdir -p ', atac.peak.path))
atac.bin.path <- "/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/ATAC/atac_matrix.binary.qc_filtered.mtx.gz"

In [27]:
system(paste0('bash bin2int_mtx.sh -z -i ', atac.bin.path, ' -o ', atac.peak.path, '/counts_int.mtx'))

In [85]:
sci.atac.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/ATAC/'
sci.atac.counts <- readMM(file.path(atac.peak.path, 'counts_int.mtx'))
sci.atac.barcodes <- readLines(file.path(sci.atac.path, 'atac_matrix.binary.qc_filtered.cells.txt'))
sci.atac.peaks <- readLines(file.path(sci.atac.path, 'atac_matrix.binary.qc_filtered.peaks.txt'))

In [86]:
sci.atac.counts <- set.names(counts = sci.atac.counts, rowname = format.peaks(sci.atac.peaks), colname = sci.atac.barcodes)

In [87]:
sci.atac.annot <- read.csv(file.path(sci.atac.path, 'cell_metadata.tissue_freq_filtered_lung.txt'))

In [88]:
any(duplicated(sci.atac.annot$cell))

[1] FALSE

In [89]:
unique(sci.atac.annot$tissue)

[1] "Lung"

In [90]:
print.names.match(sci.atac.annot$cell, colnames(sci.atac.counts))

[1] "Length1, Length2, Length intersection, identical:"
[1] "9924 81173 9924 FALSE"


In [91]:
sci.atac.counts <- sci.atac.counts[, sci.atac.annot$cell]

In [93]:
identical(colnames(sci.atac.counts), colnames(sci.ga.mat))

[1] TRUE

In [92]:
# for seurat.
sci.atac.path <- 'lung/sci-atac'
write.counts.transpose(sci.atac.counts, sci.atac.path)
write.annotation(sci.atac.annot$annotation, sci.atac.path)

Warning message in dir.create(folder, recursive = TRUE):
"'lung/sci-atac' already exists"
